In [1]:
import os
import json
import pandas as pd
import traceback


In [2]:
from langchain_community.chat_models.huggingface import ChatHuggingFace

In [3]:
from dotenv import load_dotenv


load_dotenv()  # take environment variables from .env.

True

In [4]:
KEY=os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [6]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

# chat = ChatHuggingFace(llm=llm, verbose=True)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\HP\.cache\huggingface\token
Login successful


In [7]:
messages = [
    ("system", "You are a helpful translator. Translate the user sentence to French."),
    ("human", "I love programming Python."),
]

llm.invoke(messages)

"\nAssistant: J'adore programmer en Python."

In [8]:
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [10]:
TEMPLATE = """
    Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [11]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text","number","subject","tone","response_json"],
    template = TEMPLATE
)

In [12]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

c:\Users\HP\MCQGenerator\env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [13]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [14]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [15]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [16]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"], output_variables=["quiz", "review"], verbose=True,)

In [17]:
file_path=r"C:\Users\Hp\MCQGENERATOR\data.txt"

In [18]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [19]:
TEXT

'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, 

In [20]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [21]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [22]:
try:
    # Call the chain with the input variables
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": RESPONSE_JSON  # Use the JSON string if necessary
        }
    )
    print(response)
except Exception as e:
    print(f"Error occurred: {e}")

c:\Users\HP\MCQGenerator\env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each de

In [75]:
response['quiz']

"{\n  '1': {\n    'mcq': 'All organisms are made up of cells that: a) transmit energy b) process hereditary information processed in genes c) undergo photosynthesis d) regulate their own internal environments',\n    'options': {'a': 'transmit energy', 'b': 'process hereditary information processed in genes', 'c': 'undergo photosynthesis', 'd': 'regulate their own"

In [39]:
with open(file_path, 'w') as file:
    file.write(str(response['quiz']))

In [43]:
with open(file_path, 'r') as file:
    quiz = file.read()

In [44]:
quiz = json.loads(quiz)

In [45]:

quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [46]:
quiz_table_data

[{'MCQ': 'What is the primary focus of biology?',
  'Choices': "a: The study of earth's geological features | b: The scientific study of life | c: The analysis of chemical reactions | d: The exploration of outer space",
  'Correct': 'b'},
 {'MCQ': 'Which of these is NOT a common theme in biology?',
  'Choices': 'a: Evolution | b: Energy processing | c: Regulation of internal environment | d: Manufacturing goods',
  'Correct': 'd'},
 {'MCQ': 'What type of cells do all organisms contain?',
  'Choices': 'a: Prokaryotic cells | b: Eukaryotic cells | c: Both prokaryotic and eukaryotic cells | d: Neither prokaryotic nor eukaryotic cells',
  'Correct': 'c'},
 {'MCQ': 'Which level of organization in biology includes studying the functioning of organs?',
  'Choices': 'a: Molecular biology | b: Anatomy and physiology | c: Population evolution | d: Individual organism',
  'Correct': 'b'},
 {'MCQ': 'When did life first appear on Earth?',
  'Choices': 'a: 3.7 million years ago | b: 3.7 billion year

In [47]:
quiz=pd.DataFrame(quiz_table_data)

In [48]:
quiz.to_csv("machinelearning.csv",index=False)